In [64]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing

import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

from sklearn.model_selection import train_test_split

In [65]:
def startsWithDateAndTime(s):
    pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]) ([0-9]+):([0-9][0-9]) -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

def getDataPoint(line):   
    splitLine = line.split(' - ')
    dateTime = splitLine[0]
    date, time = dateTime.split(' ') 
    message = ' '.join(splitLine[1:])
    splitMessage = message.split(': ')
    author = splitMessage[0]
    message = ' '.join(splitMessage[1:])
    return date, time, author, message

parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = '../data/Chat de WhatsApp con Vir.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline() 
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip()
        if startsWithDateAndTime(line):
            if len(messageBuffer) > 0: 
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear()
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

In [66]:
df.head()

,Date,Time,Author,Message
0,2019-12-16,20:28,Vir,Mañana nos juntamos media horita y vemos un po...
1,2019-12-16,20:28,Vir,?!
2,2019-12-16,20:28,Vir,Un poco a modo de recuperar las weekly que ten...
3,2019-12-16,21:10,Leticia Olábarri,"Te los agradecería muchísimo, la verdad"
4,2019-12-16,21:11,Leticia Olábarri,*lo


In [67]:
def transform_names(name):
    if name == "Leticia Olábarri":
        return 1
    else:
        return 0

In [68]:
df["Author"] = df["Author"].apply(transform_names)

In [69]:
df.head()

,Date,Time,Author,Message
0,2019-12-16,20:28,0,Mañana nos juntamos media horita y vemos un po...
1,2019-12-16,20:28,0,?!
2,2019-12-16,20:28,0,Un poco a modo de recuperar las weekly que ten...
3,2019-12-16,21:10,1,"Te los agradecería muchísimo, la verdad"
4,2019-12-16,21:11,1,*lo


In [70]:
X_train, X_test, y_train, y_test = train_test_split(df["Message"], df["Author"], test_size=0.33, random_state=42)

In [71]:
max_features = 10000
sequence_length = 1000

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [72]:
vectorize_layer.adapt(X_train)

TypeError: Parameter to MergeFrom() must be instance of same class: expected tensorflow.TensorShapeProto got tensorflow.TensorShapeProto.